# Alligators Don't Apologize - Exploratory Data Analysis

In [1]:
import pandas as pd

In [5]:
# Path to the data folders
raw_data_folder = '../data/RAW/'
clean_data_folder = '../data/CLEAN/'

## First Dataset: CMU Movies

In [15]:
# Load the movie metadata
CMU_movie_metadata = pd.read_csv(raw_data_folder+'CMU_Movies_Dataset/movie.metadata.tsv', delimiter='\t', header=None)
CMU_movie_metadata.columns = ['wikipedia_movie_ID', 'freebase_movie_ID', 'movie_name', 'movie_release_date', 'movie_box_office_revenue', 'movie_runtime', 'movie_languages', 'movie_countries', 'movie_genres']
CMU_movie_metadata.head(10)

# Load the character metadata
CMU_char_metadata = pd.read_csv(raw_data_folder+'CMU_Movies_Dataset/character.metadata.tsv', sep='\t', header=None)
CMU_char_metadata.columns = ['wikipedia_movie_ID', 'freebase_movie_ID', 'movie_release_date', 'character_name', 'actor_DOB', 'actor_gender', 'actor_height_meters', 'actor_ethnicity', 'actor_name', 'actor_age_at_movie_release', 'freebase_character_actor_map_ID', 'freebase_character_ID', 'freebase_actor_ID']

# Load the plot summaries
CMU_plot_summary = pd.read_csv(raw_data_folder+'CMU_Movies_Dataset/plot_summaries.txt', sep='\t', header=None)
CMU_plot_summary.columns = ['wikipedia_movie_ID', 'plot_summary']

#######

# Include the other parts of the dataset if needed

######


## Second Dataset: GVD Dataset

In [16]:
# Load the GVD dataset
GVD_data = pd.read_csv(raw_data_folder+'GVD_Dataset/2023_gvdDatabase_1_0_country.csv')

In [20]:
GVD_data.head(10)

,year,country_code,country_name,country_region,country_subregion,population,indicator,mechanism,count,rate,count_population,source_name,source_url,entry_comment
0,2004,ABW,Aruba,Americas,Caribbean,Female,Homicide intentional,All,0,0.00,51716,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2009 up...
1,2004,ABW,Aruba,Americas,Caribbean,Male,Homicide intentional,All,2,4.25,47028,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2009 up...
2,2004,ABW,Aruba,Americas,Caribbean,Total,Homicide intentional,All,2,2.03,98744,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2009 up...
3,2005,ABW,Aruba,Americas,Caribbean,Female,Homicide intentional,All,1,1.91,52454,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2010 up...
4,2005,ABW,Aruba,Americas,Caribbean,Male,Homicide intentional,All,5,10.51,47574,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2010 up...
5,2005,ABW,Aruba,Americas,Caribbean,Total,Homicide intentional,All,6,6.00,100028,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2010 up...
6,2006,ABW,Aruba,Americas,Caribbean,Female,Homicide intentional,All,1,1.89,52895,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2011 up...
7,2006,ABW,Aruba,Americas,Caribbean,Male,Homicide intentional,All,4,8.34,47935,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2011 up...
8,2006,ABW,Aruba,Americas,Caribbean,Total,Homicide intentional,All,5,4.96,100830,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2011 up...
9,2007,ABW,Aruba,Americas,Caribbean,Female,Homicide intentional,All,0,0.00,53086,UN Office On Drugs and Crime,https://dataunodc.un.org,Data collected and/or validated in the 2012 up...


## Third dataset: Kaggle Movies Dataset

In [29]:
# Load the movies metadata
Kaggle_movies_metadata = pd.read_csv(raw_data_folder+'Kaggle_Movies_Dataset/movies_metadata.csv')

# Load the credits
Kaggle_credits = pd.read_csv(raw_data_folder+'Kaggle_Movies_Dataset/credits.csv')

# Load the keywords data
Kaggle_keywords = pd.read_csv(raw_data_folder+'Kaggle_Movies_Dataset/keywords.csv')

# Load the ratings
Kaggle_ratings = pd.read_csv(raw_data_folder+'Kaggle_Movies_Dataset/ratings.csv')

/var/folders/8z/vm21_w1s46v5nvgv_mq9wlrh0000gn/T/ipykernel_36061/680486532.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  Kaggle_movies_metadata = pd.read_csv(raw_data_folder+'Kaggle_Movies_Dataset/movies_metadata.csv')
